In [1]:
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input
import matplotlib.pyplot as plt
from utils import *
from PIL import Image
import os
from os import listdir

# Pré Process

In [3]:
directory = 'images/final/'
width = 96
height = 96
channels = 4
image_size = (width, height)
input_shape = (width, height, channels)
batch_size = 32

## Whitening & Resizing

In [8]:


folders = listdir(directory)
# add to each folder the path to the folder
folders = [directory + folder for folder in folders]

for folder in folders:
    images = listdir(folder)
    for i, image in enumerate(images):
        image_path = folder + '/' + image
        img = Image.open(image_path)
        img = img.convert("RGBA")
        datas = img.getdata()

        newData = []
        for item in datas:
            # if Alpha is 0, then make RGB values 255, 255, 255, and Alpha 0
            if item[3] == 0:
                newData.append((255, 255, 255, 255))
            # if Alpha is 255, then keep the RGB values as they are, and Alpha 255
            elif item[3] == 255:
                newData.append(item)
            # if Alpha is between 0 and 255, then make RGB values 255, 255, 255, and Alpha 255
            else:
                newData.append((255, 255, 255, 255))
        # save the image with the new data
        img.putdata(newData)
        # resize the image
        img = img.resize(image_size)
        # replace the old image with the new image with same name as the old image
        # print(image_path)
        img.save(image_path)


## Pré Process Inputs CNN

In [3]:
import pandas as pd
import numpy as np


### Link w/ folder name and csv Name

In [4]:
import cv2

In [5]:
folders = listdir(directory)
image_folder_path = [directory + folder for folder in folders]

data = pd.DataFrame(columns=['image','type1','type2'])
target = pd.DataFrame(columns=['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed'])
csv = 'PokeDataset.csv'
df = pd.read_csv(csv)

# order the df by alphabetical order of Name column
df = df.sort_values(by=['Name'])

# print(df.head())


# Create a dictionary to store the associations
image_types = {}

# Iterate through the rows in the DataFrame
for index, row in df.iterrows():
    image_name = row['Name']
    # if row type1 is not nan or Null then type1 = row['Type1'] else type1 = ''
    type1 = row['Type1'] if not pd.isnull(row['Type1']) else ''
    # if row type2 is not nan or Null then type2 = row['Type2'] else type2 = ''
    type2 = row['Type2'] if not pd.isnull(row['Type2']) else ''


    image_types[image_name] = (type1, type2)





# print(image_types)
# Iterate through the folders in the image folder
for folder_name in image_folder_path:
    pokemon_name = folder_name.split('/')[-1]
    if os.path.isdir(folder_name):
        print(pokemon_name)
        for image in folder_name:
            #add to a new datadrame image, type1, type2 of the pokemon wich is the folder name use image_types dictionary
            image_path = os.path.join(folder_name, image)
            # get the image from image_path 
            image_data = cv2.imread(image_path)
            try : 
                type1, type2 = image_types[pokemon_name]
            except KeyError:
                type2 = ''
                print(pokemon_name)

         
            # extract the HP, Attack, Defense, Sp. Atk, Sp. Def, and Speed from the df DataFrame
            stats = df.loc[df['Name'] == pokemon_name, ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
            # add the data to the data and target DataFrames
            data_tmp = pd.DataFrame({'image': [image_data], 'type1': [type1], 'type2': [type2]})
            target_tmp = pd.DataFrame(stats)
            # use concat to add the data to the data and target DataFrames
            data = pd.concat([data, data_tmp], ignore_index=True)
            target = pd.concat([target, target_tmp], ignore_index=True)
            # data = data.append({'image': image_data, 'type1': type1, 'type2': type2}, ignore_index=True)
            # target = target.append(pd.Series(stats, index=target.columns), ignore_index=True)

            
print(data.shape)
print(target.shape)


Abomasnow
Abra
Absol
Accelgor
Aegislash
Aerodactyl
Aggron
Aipom
Alakazam
Alcremie
Alomomola
Altaria
Amaura
Ambipom
Amoonguss
Ampharos
Anorith
Appletun
Applin
Araquanid
Arbok
Arcanine
Arceus
Archen
Archeops
Arctovish
Arctozolt
Ariados
Armaldo
Aromatisse
Aron
Arrokuda
Articuno
Audino
Aurorus
Avalugg
Axew
Azelf
Azumarill
Azurill
Bagon
Baltoy
Banette
Barbaracle
Barboach
Barraskewda
Basculegion
Basculin
Bastiodon
Bayleef
Beartic
Beautifly
Beedrill
Beheeyem
Beldum
Bellossom
Bellsprout
Bergmite
Bewear
Bibarel
Bidoof
Binacle
Bisharp
Blacephalon
Blastoise
Blaziken
Blipbug
Blissey
Blitzle
Boldore
Boltund
Bonsly
Bouffalant
Bounsweet
Braixen
Braviary
Breloom
Brionne
Bronzong
Bronzor
Bruxish
Budew
Buizel
Bulbasaur
Buneary
Bunnelby
Burmy
Butterfree
Buzzwole
Cacnea
Cacturne
Calyrex
Camerupt
Carbink
Carkol
Carnivine
Carracosta
Carvanha
Cascoon
Castform
Caterpie
Celebi
Celesteela
Centiskorch
Chandelure
Chansey
Charizard
Charjabug
Charmander
Charmeleon
Chatot
Cherrim
Cherubi
Chesnaught
Chespin
Chewtle
C

In [6]:
df.shape

(905, 18)

In [7]:

#for folder_name in image_folder_path:
#    # find all folder with 22 images
#    if len(os.listdir(folder_name)) == 22:
#        print(folder_name)


### One Hot Encoding

In [8]:
from sklearn.preprocessing import LabelBinarizer

# initialize the label binarizer
lb = LabelBinarizer()

# create a one-hot encoding for the type1 and type2 columns
type1 = lb.fit_transform(data['type1'])
type2 = lb.fit_transform(data['type2'])


## Define the CNN

In [5]:
# define two sets of inputs: the images and the categorical inputs type1 and type2
image_input = tf.keras.Input(shape=input_shape, name='image_input')
category_input = tf.keras.Input(shape=(2,), name='category_input')

# define the first branch of the CNN -- the image branch
# downsize the image from 96x96x4 to 24x24x4
x = tf.keras.layers.Conv2D(4, (3, 3), strides=(2, 2), padding='same')(image_input)
# downsize the image from 24x24x4 to 12x12x8
x = tf.keras.layers.Conv2D(8, (3, 3), strides=(2, 2), padding='same')(x)
# downsize the image from 12x12x8 to 6x6x16
x = tf.keras.layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same')(x)
# flatten the image from 6x6x16 to 576x16
x = tf.keras.layers.Flatten()(x)

# define the second branch of the CNN -- the categorical branch
# 18 categories plus 2 of these mixed categories is 171 combinations of type1 and type2
# upsize the categorical input from 2 to attaining 256
y = tf.keras.layers.Dense(256, activation='relu')(category_input)

# merge the output of the two branches
z = tf.keras.layers.concatenate([x, y])

# define the output of the model as the output of the final Dense layer
output = tf.keras.layers.Dense(6, activation='linear')(z)

# define the model with two inputs and one output
model = tf.keras.Model(inputs=[image_input, category_input], outputs=output)

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [6]:
# plot the model
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
